In [34]:
import pandas as pd
import numpy as np
import pmdarima as pm
from prophet import Prophet
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
import warnings

warnings.filterwarnings('ignore')

In [40]:
# Function to forecast using PMDARIMA
def forecast_pmdarima(df, reference, periods):
    data = df[df['item'] == reference][["date",'sales']]
    data = df.groupby("date")["sales"].sum()
    model = pm.auto_arima(data, start_p=2, start_q=2, start_P=2, start_Q=2,
                     max_p=5, max_q=5, max_P=5, max_Q=5, seasonal=True,
                     stepwise=False, suppress_warnings=True, d = None, max_D=5, m=12,
                     scoring = "mae", test= "adf",
                     error_action='ignore')
    forecast = model.predict(n_periods=periods)
    return forecast

def forecast_pmdarima_log(df, reference, periods):
    data = df[df['item'] == reference][["date",'sales']]
    data = df.groupby("date")["sales"].sum()
    data = np.log(data)
    model = pm.auto_arima(data, start_p=2, start_q=2, start_P=2, start_Q=2,
                     max_p=5, max_q=5, max_P=5, max_Q=5, seasonal=True,
                     stepwise=False, suppress_warnings=True, d = None, max_D=5, m=12,
                     scoring = "mae", test= "adf",
                     error_action='ignore')
    forecast = np.exp(model.predict(n_periods=periods))
    return forecast

# Function to forecast using Prophet
def forecast_prophet(df, reference, periods):
    data = df[df['item'] == reference][['date', 'sales']].rename(columns={'date': 'ds', 'sales': 'y'})
    model = Prophet()
    model.fit(data)
    future = model.make_future_dataframe(periods=periods, freq='M')
    forecast = model.predict(future)
    return forecast['yhat'][-periods:].values

# Function to forecast using XGBoost Regressor
def forecast_xgboost(df, reference, periods):
    data = df[df['item'] == reference]
    data['month'] = data['date'].dt.month
    data['year'] = data['date'].dt.year
    X = data[['month', 'year']]
    y = data['sales']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    model = XGBRegressor(n_estimators=1000)
    model.fit(X_train, y_train)
    
    future_dates = pd.date_range(start=df['date'].max() + pd.DateOffset(months=1), periods=periods, freq='M')
    future_df = pd.DataFrame({'date': future_dates})
    future_df['month'] = future_df['date'].dt.month
    future_df['year'] = future_df['date'].dt.year
    X_future = future_df[['month', 'year']]
    
    forecast = model.predict(X_future)
    return forecast

# Function to find the best model and forecast
def forecast_best_model(df, reference, periods):
    df['date'] = pd.to_datetime(df['date'])
    
    pmdarima_forecast = forecast_pmdarima(df, reference, periods)
    pmdarimalog_forecast = forecast_pmdarima_log(df, reference, periods)
    prophet_forecast = forecast_prophet(df, reference, periods)
    xgboost_forecast = forecast_xgboost(df, reference, periods)

    
    # Assuming you have a way to evaluate the models, for example, using a holdout set
    # Here, we'll use the last 12 months of the data as the test set
    train_data = df[df['item'] == reference].iloc[:-12]
    test_data = df[df['item'] == reference].iloc[-12:]
    
    mape_pmdarima = mean_absolute_percentage_error(test_data['sales'], pmdarima_forecast)
    mape_pmdarimalog = mean_absolute_percentage_error(test_data['sales'], pmdarimalog_forecast)
    mape_prophet = mean_absolute_percentage_error(test_data['sales'], prophet_forecast)
    mape_xgboost = mean_absolute_percentage_error(test_data['sales'], xgboost_forecast)
    
    best_model = min(mape_pmdarima,mape_pmdarimalog, mape_prophet, mape_xgboost)
    
    if best_model == mape_pmdarima:
        return 'PMDARIMA', pmdarima_forecast, mape_pmdarima
    elif best_model == mape_prophet:
        return 'Prophet', prophet_forecast, mape_prophet
    elif best_model == mape_pmdarimalog:
        return "PMDARIMALOG", pmdarimalog_forecast, pmdarimalog_forecast
    else:
        return 'XGBoost', xgboost_forecast, xgboost_forecast



    


In [41]:
# Using the model with sales in KG for H&N
df = pd.read_csv("sales_hn_kg.csv")
df = df.rename(columns = {"Sales_Time":"date", "KG":"sales"})
df = df.sort_values(by="sales", ascending=False) 

reference_list = df['item'].unique()[:4] #for testing the model I only take the first top 4 references sold in KG.

forecast_results = {}
for reference in reference_list:
    model_name, forecast, mape = forecast_best_model(df, reference, 12)
    forecast_results[reference] = {
        'model': model_name,
        'mape': mape
    }

print(forecast_results)

19:02:17 - cmdstanpy - INFO - Chain [1] start processing
19:02:17 - cmdstanpy - INFO - Chain [1] done processing
19:03:48 - cmdstanpy - INFO - Chain [1] start processing
19:03:48 - cmdstanpy - INFO - Chain [1] done processing
19:05:11 - cmdstanpy - INFO - Chain [1] start processing
19:05:12 - cmdstanpy - INFO - Chain [1] done processing
19:06:37 - cmdstanpy - INFO - Chain [1] start processing
19:06:37 - cmdstanpy - INFO - Chain [1] done processing


{'70000102': {'model': 'XGBoost', 'mape': array([13013.065, 13218.351, 13028.387, 13020.844, 12952.863, 13034.426,
       13086.507, 13163.917, 20000.006, 13102.287, 13009.839, 13000.001],
      dtype=float32)}, '70003338': {'model': 'XGBoost', 'mape': array([1526.0441 , -923.6609 ,  205.00598,  339.29712, 2338.7273 ,
       -767.7252 ,  718.2128 ,  484.83813,  583.33484,  299.99203,
        545.8343 ,  599.9985 ], dtype=float32)}, '70002405': {'model': 'XGBoost', 'mape': array([ 2686.6357,  2836.9536,  1729.1589,  7104.645 ,  9153.81  ,
       14965.797 ,  3026.9382,  3999.5735, 18624.994 ,   837.7502,
        2661.8525,  3037.4998], dtype=float32)}, '70003427': {'model': 'XGBoost', 'mape': array([1807.5808, 2904.682 , 1691.1058, 2276.4119, 1298.6915, 1214.2831,
       1969.5535, 1307.0023, 1397.9176, 1116.6633, 2834.376 , 1679.1664],
      dtype=float32)}}
